In [1]:
import pyarrow.dataset as ds

# dataset = ds.dataset("/home/doserbd/data/gaia/xp_sampled_mean_spectrum/parquet/XpSampledMeanSpectrum_000000-003111.parquet",
dataset = ds.dataset("../tests/data/XpSampledMeanSpectrum", format="parquet")
print(dataset.to_table().schema.metadata[b"flux_shape"])

b'(1, 344)'


In [2]:
from hipster import CatalogGenerator, Inference

catalog_generator = CatalogGenerator(
    encoder=Inference("../tests/models/vae_encoder.onnx"),
    data_directory="../tests/data/XpSampledMeanSpectrum",
)

catalog = catalog_generator()
catalog

2025-01-25 11:41:59.328376709 [W:onnxruntime:, graph.cc:109 MergeShapeInfo] Error merging shape info for output. 'encoder_1' source:{} target:{1}. Falling back to lenient merge.


,source_id,latent_position,RA2000,DEC2000
0,38655544960,"[-0.14075603, 0.005951977, 0.9900264]",177.578644,81.901123
1,2851858288640,"[0.85242873, -0.47868976, -0.21028858]",330.683197,-12.139267
2,3332894779520,"[-0.43773812, -0.14929375, 0.886621]",198.832306,62.451668
3,3371550165888,"[-0.51504326, -0.08609367, 0.8528296]",189.489731,58.520782
4,4711579935744,"[-0.02911299, -0.841445, -0.53955793]",268.018402,-32.653557
5,6223408420864,"[-0.58074635, -0.18765895, 0.7921603]",197.907425,52.387852
6,6944962925824,"[-0.5933531, -0.75791544, -0.2711021]",231.943497,-15.729858
7,7284264691456,"[0.45781064, 0.8837401, 0.09701948]",62.614147,5.567566
8,7632157690368,"[-0.71567076, -0.21168122, 0.6655872]",196.477173,41.727394
9,13301514552192,"[0.7969228, 0.056911614, -0.6013943]",4.084798,-36.969826


In [3]:
from astropy.io.votable import writeto
from astropy.table import Table

table = Table.from_pandas(catalog)
writeto(table, "catalog.vot")

In [4]:
import numpy as np
flux = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]], dtype=np.float32)

for i, x in enumerate(flux):
    flux[i] = (x - x.min()) / (x.max() - x.min())

flux

array([[[0. , 0.2, 0.4],
        [0.6, 0.8, 1. ]],

       [[0. , 0.2, 0.4],
        [0.6, 0.8, 1. ]]], dtype=float32)